# Sanity checks
Some group-level analyses.

In [1]:
import numpy as np
import pandas as pd
import joblib as jl
import nibabel as nib
import os.path as op
import matplotlib.pyplot as plt
from glob import glob
from nilearn import plotting, masking, image
from nistats.first_level_model import FirstLevelModel
from nistats.second_level_model import SecondLevelModel
from nilearn.datasets import load_mni152_template
from nistats.thresholding import map_threshold

%matplotlib inline

mni = load_mni152_template()

/opt/conda/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


## Working memory

In [22]:
def run_firstlevels(task, space='MNI152NLin2009cAsym', conf_cols=None, ricor=True, n_jobs=1, n_subs=None, **flm_kwargs):

    if conf_cols is None:
        conf_cols = []
        
    BIDS_DIR = '../..'
    FP_DIR = op.join(BIDS_DIR, 'derivatives', 'fmriprep')
    PHYS_DIR = op.join(BIDS_DIR, 'derivatives', 'physiology')
    
    funcs = sorted(glob(op.join(FP_DIR, 'sub-*', 'func', f'*task-{task}*space-{space}*bold.nii.gz')))
    masks = sorted(glob(op.join(FP_DIR, 'sub-*', 'func', f'*task-{task}*space-{space}*brain_mask.nii.gz')))
    confs = sorted(glob(op.join(FP_DIR, 'sub-*', 'func', f'*task-{task}*regressors.tsv')))
    ricors = sorted(glob(op.join(PHYS_DIR, 'sub-*', 'physio', f'*task-{task}*regressors.tsv')))
    
    all_mri_subs = [op.basename(f).split('_')[0] for f in funcs]
    
    all_ricor_subs = [op.basename(f).split('_')[0] for f in ricors]
    all_compl_subs = list(set(all_mri_subs) & set(all_ricor_subs))

    funcs = [f for f in funcs if op.basename(f).split('_')[0] in all_compl_subs]
    masks = [f for f in masks if op.basename(f).split('_')[0] in all_compl_subs]
    confs = [f for f in confs if op.basename(f).split('_')[0] in all_compl_subs]
    ricors = [f for f in ricors if op.basename(f).split('_')[0] in all_compl_subs]
    
    if n_subs is None:
        n_subs = len(confs)
    
    if 'hrf_model' not in flm_kwargs.keys():
        flm_kwargs['hrf_model'] = 'glover'
        
    if 'noise_model' not in flm_kwargs.keys():
        flm_kwargs['noise_model'] = 'ols'
        
    if 'smoothing_fwhm' not in flm_kwargs.keys():
        flm_kwargs['smoothing_fwhm'] = 5
    
    flms = jl.Parallel(n_jobs=n_jobs)(jl.delayed(_fit_parallel)(
            func, mask, conf, ricor, conf_cols, **flm_kwargs
        ) for func, mask, conf, ricor in zip(funcs[:n_subs], masks[:n_subs], confs[:n_subs], ricors[:n_subs])
    )
    
    return flms

def _fit_parallel(func, mask, conf, ricor, conf_cols, **flm_kwargs):
    print("Fitting %s" % op.basename(func))   

    sub_ids = [op.basename(f).split('_')[0] for f in [func, mask, conf]]
    if len(set(sub_ids)) != 1:
        raise ValueError('Not aligned!')
        
    conf = pd.read_csv(conf, sep='\t')
    cols2idx = [col for col in conf.columns if 'cosine' in col]
    conf = conf.loc[:, cols2idx + conf_cols + ['trans_x', 'trans_y', 'trans_z', 'rot_x', 'rot_y', 'rot_z']]
    conf = conf.fillna(0)
    
    ricor_df = pd.read_csv(ricor, sep='\t')
    conf = pd.concat((conf, ricor_df), axis=1)
    conf['icept'] = 1

    flm = FirstLevelModel(
        t_r=nib.load(func).header['pixdim'][4],
        drift_model=None,
        mask_img=mask,
        **flm_kwargs
    )
    
    flm.fit(func, design_matrices=conf)
    return flm

def run_grouplevel(flms, contrast, hack=True, second_level_stat_type=None, output_type='z_score'):

    filt_flms = []
    
    if hack:
        str_contrast = ''.join([i for i in contrast])
    else:
        str_contrast = ''.join([i for i in contrast if not i.isdigit()])

    cols = str_contrast.replace('*', '').replace('-', '').replace('+', '').strip().split(' ')
    cols = [col for col in cols if col]
    for flm in flms:
        dm_cols = flm.design_matrices_[0].columns.tolist()

        if all(col in dm_cols for col in cols):
            filt_flms.append(flm)

    print("Found %i FLMs with the specified contrast." % len(filt_flms))
    slm = SecondLevelModel()
    slm.fit(filt_flms)
    img = slm.compute_contrast(
        first_level_contrast=contrast,
        second_level_stat_type=second_level_stat_type,
        output_type=output_type
    )
    return img

In [27]:
flms = run_firstlevels(
    task='moviewatching', space='MNI',
    conf_cols=None,
    ricor=True,
    n_jobs=5,
    n_subs=90
)

In [28]:
cardiac_cols = [
    'cardiac_cos_00', 'cardiac_sin_00', 'cardiac_cos_01', 'cardiac_sin_01', 'cardiac_cos_02', 'cardiac_sin_02'
]
resp_cols = [
    'resp_cos_00', 'resp_sin_00', 'resp_cos_01', 'resp_sin_01', 'resp_cos_02', 'resp_sin_02', 'resp_cos_03', 'resp_sin_03'
]

interact_cols = [
    'interaction_add_cos_00', 'interaction_add_sin_00', 'interaction_diff_cos_00', 'interaction_diff_sin_00'
]
ricor_cols = cardiac_cols + resp_cols + interact_cols

In [31]:
conf_img = run_grouplevel(flms, '+ '.join(ricor_cols), hack=True, second_level_stat_type='F')
#conf_img = run_grouplevel(flms, 'rvt', hack=True, second_level_stat_type='F')
#conf_img = image.threshold_img(conf_img, 1.7)
plotting.view_img(conf_img)

Found 90 FLMs with the specified contrast.


/opt/conda/lib/python3.7/site-packages/nilearn/plotting/js_plotting_utils.py:103: UserWarning: It seems you have created more than 10 nilearn views. As each view uses dozens of megabytes of RAM, you might want to delete some of them.
  warnings.warn('It seems you have created more than 10 '
